In [ ]:
import numpy as np
import joblib
from mne.stats import spatio_temporal_cluster_1samp_test
from scipy import stats
from mne import (pick_events, find_events, read_source_spaces, spatial_src_adjacency, 
                 make_forward_solution, compute_rank, compute_raw_covariance, pick_info)
from mne.io import read_info
from mne import Epochs
from mne.minimum_norm import make_inverse_operator
from mne.preprocessing import ICA
from os.path import join, isfile
from os import listdir
import glob
from mne.minimum_norm import apply_inverse
from mne.epochs import equalize_epoch_counts
from mne.preprocessing import corrmap
import utils.util_functions as ut

# PREPROCESSING
### Compute ICA weights

In [ ]:
def compute_ica(subject_id, raw_dir, ica_dir):

    joblib.load(join(raw_dir, subject_id))
    # compute ica on concatenated data (all 6 blocks)
    raw_copy = subject_id.copy().filter(l_freq=1, h_freq=None)
    raw_filt = raw_copy.resample(sfreq=250)
    raw_filt.set_annotations(None)
    
    ica = ICA(n_components=None, max_iter='auto', random_state=97)
    ica.fit(raw_filt)
    #plot 
    fig= ica.plot_components(picks=range(0,20))
    fig.savefig(join(subject_id +'.png'))
    
    #save ica
    joblib.dump(ica, join(ica_dir, subject_id))    
        
    return ica

### Use CORRMAP to identify and remove bad components

In [ ]:
raw_dir= 'raw_dir' # path with concatenated raw data
ica_dir= 'ica_dir' # path with ica weights

all_subjects= listdir(raw_dir)

icas=[]
for sub in all_subjects:
    # load ica weights
    ica= joblib.load(join(ica_dir, sub))
    icas.append(ica)
  
del ica
    
#  I choose subject 19860620hrwc as template for eyemov 
subject_id='19860620hrwc'

# load raw file and downsample it 
raw= joblib.load(join(raw_dir, subject_id))
#raw = raw.resample(sfreq=250)

template= all_subjects.index(subject_id)
raw.load_data()
icas[template].plot_sources(raw)

# EYES
eog_inds, eog_scores = icas[template].find_bads_eog(raw, reject_by_annotation=False)
corrmap(icas, template=(template, eog_inds[0]), ch_type='mag', plot=False)

# check for the correct threshold which include all of the partecipant
threshold=0.85 # reduce the value untill you have "At least 1 IC detected for each subject." as output
corrmap(icas, template=(template, eog_inds[0]), ch_type='mag', threshold=threshold, plot=False)

# now mark them in label as eyemovements
corrmap(icas, template=(template, eog_inds[0]), ch_type='mag', threshold=threshold, label='eyemov')

# plot topography for each marked components and if you have doubts load raw file and visually inspect

for ii, ica in enumerate(icas):
  icas[ii].plot_components(picks=icas[ii].labels_['eyemov'])

# now do the same for heartbeat
# HEARTBEAT
# I choose subject 8: 19860620hrwc as template for eyemov
subject_id='19921030jard'
raw= joblib.load(join(raw_dir, subject_id))
raw = raw.resample(sfreq=250)

template= all_subjects.index(subject_id)
raw.load_data()

ecg_inds, ecg_scores = icas[template].find_bads_ecg(raw, reject_by_annotation=False)
icas[template].plot_sources(raw)

corrmap(icas, template=(template, ecg_inds[0]), ch_type='mag', plot=False)

threshold=0.6 # reduce the value untill you have "At least 1 IC detected for each subject." as output
corrmap(icas, template=(template, ecg_inds[0]), ch_type='mag', threshold=threshold, plot=False)

corrmap(icas, template=(template, ecg_inds[0]), ch_type='mag', threshold=threshold, label='heart')

for ii, ica in enumerate(icas):
  ica.plot_components(picks=icas[ii].labels_['heart'])

# add components to exclude and check them
for ii, ica in enumerate(icas):
  icas[ii].exclude=[]
  icas[ii].exclude = icas[ii].labels_['eyemov']
  icas[ii].exclude.extend(icas[ii].labels_['heart'])
  
dict_icas= {}
for ii, sub in enumerate(all_subjects):
  
  dict_icas.update({sub: icas[ii]})

joblib.dump(dict_icas, join(ica_dir, 'all_icas')) 


# Get info on removed components to report
badcomps=[]
for sub in all_subjects:
  
  badcomps.append(len(icas[sub].exclude))

avg_num_icas= round(np.mean(badcomps), 2)
std_num_icas= round(np.std(badcomps), 2)
range_num_inca= [np.min(badcomps), np.max(badcomps)]

### Apply ICA weigths on raw unfiltered data

In [ ]:
def apply_ica(subject_id, raw_dir, ica_dir, out_dir):
        
        # load concatenated raw data     
        raw= joblib.load(join(raw_dir, subject_id))
        
        # load ica weigths
        icas= joblib.load(join(ica_dir, 'all_icas'))
        
        # apply ica weights
        ica= icas[subject_id]
        ica.apply(raw)
        
        #save preprocessed data
        joblib.dump(raw, join(out_dir, subject_id))    

# SOURCE SPACE ANALYSIS
### Compute forward and inverse solution

Note: Before using the following function, use coregistration script in utils to create headmodels starting from fsaverage template

In [ ]:
def source_trans(subject_id, raw_path, fs_path, ftrans, save_path):
    
        # define src and bem files
        src = join(fs_path, 'fsaverage-ico-4-src.fif')
        bem = join(fs_path, 'fsaverage-5120-5120-5120-bem-sol.fif')
        
        # use individual coregistrations   
        trans =  join(ftrans, subject_id + '-trans.fif')  
        
        fname = glob.glob(join(raw_path, '**/%s_block1_trans_sss.fif' % subject_id),
                            recursive=True)

        # get info for MNE from a random dataset
        info = read_info(fname[0], verbose=False)
        info = pick_info(info, pick_types(info, meg=True))
        
        # compute noise cov on empty room
        empty_room = ut.get_nearest_empty_room(info)

        empty_room = ut.preproc_data(empty_room, max_filt=True, coord_frame='meg', notch=False,
                                  apply_filter=True, hp_lp_freqs=(0.1, 40.), do_downsample=True,
                                  resample_freq=250.)

        noise_cov = compute_raw_covariance(empty_room, rank=None, picks='meg')
       
        true_rank = compute_rank(noise_cov, info=empty_room.info)  # inferring true rank

        # make forward solution for MNE
        fwd = make_forward_solution(info=info, trans=trans, src=src, bem=bem, meg=True, eeg=False)

        # compute inverse operator for MNE
        inv = minimum_norm.make_inverse_operator(info, fwd, noise_cov, rank=true_rank)  # rank=None

        data2save = {'inv': inv, 'fwd': fwd, 'noise_cov': noise_cov, 
                     'empty_room_info': empty_room.info}

        joblib.dump(data2save, join(save_path, subject_id + '.dat'))

# Epoching, computing ERFs, and apply inverse solution to compute source timecourses (stc)

In [ ]:
def epoching_source(data, tmin, tmax, baseline, decim):
    starting_trigger_list= [48, 80, 88, 104]
    conditions= ['L40', 'L478', 'S40', 'S478']
    events_all= find_events(data)
    
    epoched_all={}
    # first: epoching according to condition
    for c, idx in enumerate(starting_trigger_list):
      pos= np.where((events_all==idx))[0]
      start= ((events_all[pos,0]-100)/1000)-(data.first_samp/data.info['sfreq'])
      ending= start+327
      first_epoch= data.copy().crop(tmin= start[0], tmax= ending[0], include_tmax= True)
      events= mne.find_events(first_epoch)
        
      # second: create bsl corrected epochs for erps (epoch novel)
      ev= pick_events(events, include= 64)
      event_id= {'novel': 64}        
      picks = pick_types(first_epoch.info, meg=True)    
      
      if decim== None:
        decim= 1 

      epochs= mne.Epochs(first_epoch, ev, event_id, tmin= tmin, tmax= tmax, 
                   baseline= baseline, decim= decim, picks=picks, reject=None, 
                   reject_by_annotation=True, preload=True)
        
      epoched_all.update({conditions[c]: epochs})
    
    return epoched_all

In [ ]:
def compute_stcs(cond, subject_id, data_path, out_dir, src_path):
        
    if cond== 40:
        c= ['L40', 'S40']
    elif cond == 478:
        c= ['L478', 'S478']

    # parameters for inverse solution
    snr = 3.0
    lambda2 = 1.0 / snr ** 2
    method = "MNE" 
        
    # Load source space for this subject 
    src= joblib.load(join(src_path, subject_id + '.dat'))
    inverse_operator= src['inv']
        
    # Load epochs for novel sounds

    epochs_nov= joblib.load(join(data_path, 'nov_' + subject_id))
        
    # select the condition and cut pre-stimulus period
        
    epochs1= epochs_nov[c[0]]
    epochs2= epochs_nov[c[1]]
    equalize_epoch_counts([epochs1, epochs2])  
        
    # compute evoked response
    evoked1 = epochs1.average()
    evoked2 = epochs2.average()
        
    # cropping t < 0 
    evoked1.crop(0, None)
    evoked2.crop(0, None)
                         
    # Apply inverse operator  
    stcs1 = apply_inverse(evoked1, inverse_operator, lambda2, method)
    stcs2 = apply_inverse(evoked2, inverse_operator, lambda2, method)
        
    joblib.dump(stcs1, join(out_dir, 'stcs' + '_' +c[0] + '_' + subject_id))  
    joblib.dump(stcs2, join(out_dir, 'stcs' + '_' +c[1] + '_' + subject_id))

# STATISTICS
### Make EELBRAIN DATASET

In [ ]:
import eelbrain as eb

stc_dir= 'SOURCE/stcs'
mri_avg= '/freesurfer/'
data_path= join('SOURCE/epoched_novel/')

stcs_list= [ x for x in sorted(listdir(stc_dir)) if "stcs" in x ]
all_subjects = sorted(listdir(data_path))   


data4ds = []
for condition in stcs_list:

        stcs= joblib.load(join(stc_dir, condition))
    
        idx_cond= condition.replace('stcs_', '')
    
        for ii, sub in enumerate(stcs):

            stc= eb.load.fiff.stc_ndvar(sub, src= 'ico-4', subject='fsaverage', subjects_dir=mri_avg,
                       method= 'MNE', name= 'stc')
    
            subject_id= all_subjects[ii]
            
            data4ds.append([subject_id, idx_cond, stc])

ds = eb.Dataset.from_caselist(['subject_id', 'condition', 'stc'],
                                  data4ds, random='subject_id')


In [ ]:

# Perform clusterbased permutation
duration= '40' # or 478
ds_plot= ds.sub(((ds["condition"]== "S" + duration) 
                         | (ds["condition"]== "L" + duration)))

ttest = eb.testnd.TTestRelated("stc", "condition", match="subject_id", 
    ds=ds_plot, tail= 0,  tfce=True,  tstart=0, tstop=0.499)
